In [43]:
import requests
import pandas as pd
import datetime


In [44]:
def get_value_and_datetime(url):
    r=requests.get(url)
    data = r.json()
    data_value_list = data["value"]
    df_raw = pd.DataFrame(data_value_list)
    df_raw["datetime"]=df_raw["date"].apply(transform_milli_to_datetime)
    return df_raw[["datetime","value"]]

def get_value_and_date(url):
    r=requests.get(url)
    data = r.json()
    data_value_list = data["value"]
    df_raw = pd.DataFrame(data_value_list)
    df_raw["date"]=df_raw["ref"]
    return df_raw[["date","value"]]


def transform_milli_to_datetime(milli):
    return datetime.datetime.fromtimestamp(milli/1000)

In [45]:
#create period variable
period = "latest-months"

url_temp    = "https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/1/station/161910/period/"+period+"/data.json"
url_vind    = "https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/4/station/161910/period/"+period+"/data.json"
url_neder   = "https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/5/station/161910/period/"+period+"/data.json"
url_sikt     = "https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/12/station/161910/period/"+period+"/data.json"

# 1	Lufttemperatur	momentanvärde, 1 gång/tim
# 4	Vindhastighet	medelvärde 10 min, 1 gång/tim
# 10	Solskenstid	summa 1 timme, 1 gång/tim
# 12	Sikt	momentanvärde, 1 gång/tim

In [46]:
df_temp = get_value_and_datetime(url_temp)
df_vind = get_value_and_datetime(url_vind)
df_sikt = get_value_and_datetime(url_sikt)
df_neder = get_value_and_date(url_neder)

df_temp.rename(columns = {'value':'temperatur'}, inplace = True)
df_vind.rename(columns = {'value':'vindhastighet'}, inplace = True)
df_sikt.rename(columns = {'value':'sikt'}, inplace = True)
df_neder.rename(columns = {'value':'nederbörd'}, inplace = True)

pd_comb = pd.merge(df_temp, df_vind, how="outer", on="datetime")
pd_comb = pd.merge(pd_comb, df_sikt, how="outer", on="datetime")

pd_comb.rename(columns = {'datetime':'ObsDateTime'}, inplace = True)
df_neder.rename(columns = {'date':'ObsDate'}, inplace = True)


#insert period as column
pd_comb["period"]=period
df_neder["period"]=period

#insert current time as column
pd_comb["ReadInTime"]=datetime.datetime.now()
df_neder["ReadInTime"]=datetime.datetime.now()


In [47]:
#read pd_comb into sql server database
from sqlalchemy import create_engine
engine = create_engine('mssql+pyodbc://localhost/SMHI?driver=SQL+Server+Native+Client+11.0')
pd_comb.to_sql('stg_Väder_DateTime', engine, if_exists="replace", index=False)
df_neder.to_sql('stg_Väder_Date', engine, if_exists='replace', index=False)


128

In [51]:
#run proc in sql server
import pyodbc
conn = pyodbc.connect('Driver={SQL Server};'
                        'Server=localhost;'
                        'Database=SMHI;'
                        'Trusted_Connection=yes;')

cursor = conn.cursor()
cursor.execute('Clean_Väder_DateTime')
cursor.execute('Clean_Väder_Date')
conn.commit()
cursor.close()
conn.close()
